# Importamos librerías y conectamos al drive

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk
from nltk.tokenize import word_tokenize 
from   nltk.tokenize import TreebankWordTokenizer 
from   nltk.stem     import SnowballStemmer, WordNetLemmatizer #PorterStemmer
from   nltk.corpus   import stopwords #lista de stop words

nltk.download('wordnet') #me bajo los datos para poder usarlos
nltk.download('punkt')
nltk.download('stopwords')

tokenizer  = TreebankWordTokenizer()
stemmer    = SnowballStemmer("english") #PorterStemmer
lemmatizer = WordNetLemmatizer()

 #Importo los vectorizadores
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from collections import Counter
import pickle #para guardado de datos
from sklearn.naive_bayes             import MultinomialNB
import os
!pip install varname
from varname import nameof #para obtener el nombre de variables
from sklearn.model_selection import cross_val_score

# Imports para creación del Modelo:
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import Sequential
from keras import utils as np_utils

# Imports para la transformación de Labels:
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import precision_score,recall_score,f1_score,roc_auc_score, make_scorer,confusion_matrix
from sklearn.model_selection import cross_validate

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#Guardado y Cargado de archivos

In [ ]:
#guardado de archivos
def save_txt(file,name,path='/content/drive/Shareddrives/Redes neuronales/TP1/data_results/'):
  path_name = f'{path}{name}.txt'
  with open(path_name, 'wb') as fp:
    pickle.dump(file, fp)

In [ ]:
#cargar archivos guardados
def open_txt(file,path='/content/drive/Shareddrives/Redes neuronales/TP1/data/'):
  name = f'{path}{file}.txt'
  with open(name, 'rb') as fp:
    file = pickle.load(fp)
  return file

#Funciones de preprocesamiento de texto

In [ ]:
#procesamiento de todos los articulos:
def procesamiento_textos(text, text_name,is_lc=True,is_lem=True,is_stop=True,is_stem=True,is_alpha=True,path='/content/drive/Shareddrives/Redes neuronales/TP1/data_results/'):
  text_filtrado = list()
  text_filtrado = [" ".join(procesamiento_texto(x,index,is_lc,is_lem,is_stop,is_stem,is_alpha)) for index,x in enumerate(text)]
  
  #nombre para guardado de archivo
  name = f'{text_name}_filtrado'#nameof(text)
  name += ('_lc' if is_lc==True else '')
  name += ('_lem' if is_lem==True else '')
  name += ('_stop' if is_stop==True else '')
  name += ('_stem' if is_stem==True else '')
  name += ('_alpha' if is_alpha==True else '')

  save_txt(text_filtrado,name,path) #guardamos textos procesado
  return text_filtrado

In [ ]:
#procesamiento de cada texto
def procesamiento_texto(text,index,is_lc=True,is_lem=True,is_stop=True,is_stem=True,is_alpha=True):
  # if index%100 == 0:
  #   print(index)
  text_lc = [text.lower() if is_lc==True else text] #all text in lower case
  tok = word_tokenize(text_lc[0]) #divide en palabras, lista de strings
  lem = [[lemmatizer.lemmatize(x,pos='v') for x in tok] if is_lem==True else tok] #pos='v' verbo, 'n':noun, 'a':adjetivo
  stop = [[x for x in lem[0] if x not in stopwords.words('english')] if is_stop==True else lem[0]] #elimina stop words,ejemplo: prepos, articulos
  stem = [[stemmer.stem(x) for x in stop[0]] if is_stem==True else stop[0]] #va a la raiz de las palabras
  alpha = [[x for x in stem[0] if x.isalpha()] if is_alpha==True else stem[0]] #elimino lo que no son palabras, ej:comas,puntos
  
  return alpha[0]

#Funciones NBMN

In [ ]:
def NBMN_crossval_score(train,labels_train,tf_idf_=True,alpha_ = 1,min_df_=0.000001, max_df_=0.8, ngram_range_ = (1,2),cross_sets = 5):

  CV = (TfidfVectorizer if tf_idf_ is True else CountVectorizer)(min_df=min_df_, max_df=max_df_, ngram_range = ngram_range_)
  cv_train = CV.fit_transform(train)

  clf = MultinomialNB(alpha=alpha_, class_prior=None, fit_prior=False) #construyo modelo

  scores = cross_val_score(clf, cv_train, labels_train, cv=cross_sets)
  score = scores.mean()
  
  return score

In [ ]:
def crossval_score_parameters_preprocessing(train,labels_train,text_name='train_valid',is_lc_=True,is_lem_=True,is_stop_=True,is_stem_=True,is_alpha_=True,tf_idf_=True,alpha_ = 1,min_df_=0.000001, max_df_=0.8, ngram_range_ = (1,2),cross_sets = 5,path_='/content/drive/Shareddrives/Redes neuronales/TP1/data_results/'):
  print("preprocessing....")
  text_train_filtrado = procesamiento_textos(train,text_name,is_lc=is_lc_,is_lem=is_lem_,is_stop=is_stop_,is_stem=is_stem_,is_alpha=is_alpha_,path=path_)
  print("evaluating model....")
  score = NBMN_crossval_score(text_train_filtrado,labels_train,tf_idf_,alpha_,min_df_, max_df_, ngram_range_ ,cross_sets)
  return score

In [ ]:
def crossval_score_parameters_preprocessing_loaded(train,labels_train,text_name='train_valid',is_lc_=True,is_lem_=True,is_stop_=True,is_stem_=True,is_alpha_=True,tf_idf_=True,alpha_ = 1,min_df_=0.000001, max_df_=0.8, ngram_range_ = (1,2),cross_sets = 5,path='/content/drive/Shareddrives/Redes neuronales/TP1/data_results/'):
  print("loading preprocessed text....")
  #nombre para guardado de archivo
  name = f'{text_name}_filtrado'#nameof(text)
  name += ('_lc' if is_lc_==True else '')
  name += ('_lem' if is_lem_==True else '')
  name += ('_stop' if is_stop_==True else '')
  name += ('_stem' if is_stem_==True else '')
  name += ('_alpha' if is_alpha_==True else '')

  text_train_filtrado = open_txt(name,path)
  print("evaluating model....")
  score = NBMN_crossval_score(text_train_filtrado,labels_train,tf_idf_,alpha_,min_df_, max_df_, ngram_range_,cross_sets)
  return score

In [ ]:
##ver si esta
def run_sklearn(train_,labels_train,valid_,labels_valid,hp,test=False):

  train = train_.copy()
  valid = valid_.copy()
  CV = (TfidfVectorizer if hp['tf_idf'] is True else CountVectorizer)(min_df=hp['min_df'], max_df=hp['max_df'], ngram_range = (1,2))
  cv_train = CV.fit_transform(train)
  cv_valid = CV.transform(valid)
  
  clf = MultinomialNB(alpha=hp['alpha'], class_prior=None, fit_prior=False) #construyo modelo

  print('training model...')
  clf.fit(cv_train,labels_train)

  print('evaluating hyperparameters...')
  score = clf.score(cv_valid, labels_valid)

  return score

In [ ]:
def run_sklearn_cross_val(train_,labels_train,hp,cross_sets = 5):

  train = train_.copy()
  CV = (TfidfVectorizer if hp['tf_idf'] is True else CountVectorizer)(min_df=hp['min_df'], max_df=hp['max_df'], ngram_range = (1,2))
  cv_train = CV.fit_transform(train)
  
  clf = MultinomialNB(alpha=hp['alpha'], class_prior=None, fit_prior=False) #construyo modelo

  print('evaluating hyperparameters...')

  scores = cross_val_score(clf, cv_train, labels_train, cv=cross_sets)
  score = scores.mean()

  return score

In [ ]:
def run_NBMN_test(train_,labels_train,valid_,labels_valid,text_test,tf_idf_=True,alpha_ = 1,min_df_=0.000001, max_df_=0.8, ngram_range_ = (1,2)):

  train = train_.copy()
  valid = valid_.copy()
  CV = (TfidfVectorizer if tf_idf_ is True else CountVectorizer)(min_df=min_df_, max_df=max_df_, ngram_range = (1,2))
  cv_train = CV.fit_transform(train)

  cv_valid = CV.transform(valid)
  
  clf = MultinomialNB(alpha=alpha_, class_prior=None, fit_prior=False) #construyo modelo

  clf.fit(cv_train,labels_train)
  score = clf.score(cv_valid, labels_valid)

  cv_test = CV.transform(text_test)
  test_labels = clf.predict(cv_test)

  df_test = pd.DataFrame(data=test_labels, columns=["pred_labels"],)
  df_test.index.names = ["pairID"]

  df_test.to_csv("submission.csv")

  return df_test

##Obtención de métricas

In [ ]:
#para obtener todas las métricas
def obtain_metrics(train_,labels_train,tf_idf_=True,alpha_ = 1,min_df_=0.000001, max_df_=0.8, ngram_range_ = (1,2)):

  train = train_.copy()

  CV = (TfidfVectorizer if tf_idf_ is True else CountVectorizer)(min_df=min_df_, max_df=max_df_, ngram_range = ngram_range_)
  cv_train = CV.fit_transform(train)
  
  clf = MultinomialNB(alpha=alpha_, class_prior=None, fit_prior=False) #construyo modelo

  precision_scorer = make_scorer(precision_score, average='macro')
  recall_scorer = make_scorer(recall_score, average='macro')
  f1_scorer = make_scorer(f1_score, average='macro')
  roc_auc_scorer = make_scorer(roc_auc_score, multi_class='ovr',needs_proba=True) #ovr

  secondary_metrics = ['accuracy',precision_scorer, recall_scorer, f1_scorer,roc_auc_scorer]
  metrics_results = pd.DataFrame(columns=["Accuracy","Precision","Recall","F1 score","ROC_AUC"],)
  for i in range(len(secondary_metrics)):
    score_metric = cross_val_score(clf, cv_train, labels_train, cv=5,scoring=secondary_metrics[i]).mean() 
    metrics_results[metrics_results.columns[i]] = [score_metric]

  return metrics_results

##Predecir resultados

In [ ]:
#para predecir los labels
def predict_label(train_,labels_train,text_test,tf_idf_=True,alpha_ = 1,min_df_=0.000001, max_df_=0.8, ngram_range_ = (1,2),scoring_metric = 'accuracy'):

  CV = (TfidfVectorizer if tf_idf_ is True else CountVectorizer)(min_df=min_df_, max_df=max_df_, ngram_range = ngram_range_)
  cv_train = CV.fit_transform(train_)
  cv_test = CV.transform(text_test)
    
  clf = MultinomialNB(alpha=alpha_, class_prior=None, fit_prior=False) #construyo modelo

  results = cross_validate(clf,cv_train,labels_train, cv=5,scoring=scoring_metric, return_estimator = True)

  df_test = pd.DataFrame(columns=["1","2","3","4","5"],)
  df_test.index.names = ["pairID"]
  models = results['estimator']
  index = 0
  for model in models:
    y_pred = model.predict(cv_test)
    df_test[df_test.columns[index]] = y_pred
    index += 1
  df_test['pred_labels'] = df_test.mode(axis=1)[0]
  
  df_test['pred_labels'].to_csv("submission.csv") #guardado de labels predichos

  return df_test

# Funciones MLP

In [ ]:
def count_vectorizer_MLP(train, valid, min_df_=0.000001, max_df_=0.8, ngram_range_ = (1,2)):
  cv = CountVectorizer(min_df=min_df_, max_df=max_df_, ngram_range = ngram_range_)
  cv_train = cv.fit_transform(train)
  cv_valid = cv.transform(valid)
  return cv_train, cv_valid

In [ ]:
# Fiteamos el OneHot
def labels_transform(labels_train, labels_val):
  ohe = OneHotEncoder()
  ohe.fit(np.array(labels_train).reshape(-1,1))
  labels_train_enc = ohe.transform(np.array(labels_train).reshape(-1,1))
  labels_valid_enc = ohe.transform(np.array(labels_val).reshape(-1,1))
  return labels_train_enc, labels_valid_enc

In [ ]:
# Se usa un generador de datos:
class DataGenerator(np_utils.all_utils.Sequence):
  'Generates data for Keras'
  def __init__(self, X_data, labels, batch_size=32, n_channels=1, shuffle=True):
    'Initialization'
    # Propiedades:
    self.batch_size = batch_size
    self.labels = labels
    self.X_data = X_data
    self.n_channels = n_channels
    self.shuffle = shuffle
    self.on_epoch_end()

  def __len__(self):
    'Denotes the number of batches per epoch'
    samples_per_epoch = self.X_data.shape[0]
    return int(np.floor(samples_per_epoch / self.batch_size))


  def __getitem__(self, counter):
    'Generate one batch of data'
    # Generate indexes of the batch
    indexes = self.indexes[counter*self.batch_size:(counter+1)*self.batch_size]
    # Generate data
    X, y = self.__data_generation(indexes)
    return X, y


  def on_epoch_end(self):
    # Este metodo será llamado al principio de la clase y una vez al final de cada epoch.
    'Updates indexes after each epoch'
    self.indexes = np.arange(self.X_data.shape[0]) # Acá declaro mis indexes
    if self.shuffle == True:
      # Shuffleo mis indexes
      np.random.shuffle(self.indexes)


  def __data_generation(self, indexes):
    # Produce el batche de datos en si.
    'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
    # Initialization
    X_batch = self.X_data[indexes,:].todense()
    y_batch = self.labels[indexes,:].todense()
    return np.array(X_batch), np.array(y_batch)

In [ ]:
def MLP_model(cant_capas,cv_train, cant_neuronas = [3], activation_func = ["softmax"], learning_rate = 0.01, alpha = None, loss = "categorical_crossentropy", metrics = ["accuracy"]):
  model = Sequential()
  max_capas = 0
  if len(activation_func) == cant_capas and len(cant_neuronas) == cant_capas:
    while cant_capas > max_capas:
    # Voy agregando capas, a medida que se necesite
      if max_capas == 0:
        if activation_func[max_capas] == "LeakyReLU":
          layer1 = tf.keras.layers.Dense(cant_neuronas[max_capas], activation= tf.keras.layers.LeakyReLU(alpha=alpha[max_capas]),  input_shape = (cv_train.shape[1], ))
          #model = tf.keras.Sequential([layer1])
          model.add(layer1)
          #print('entre al LeakyRelu', max_capas)
        else:
          #print('entre al else', max_capas)
          model.add(Dense(cant_neuronas[max_capas], activation = activation_func[max_capas], input_shape = (cv_train.shape[1], )))
      else:
        #print('entra 2')
        if activation_func[max_capas] == "LeakyReLU":
          #print('volvio a entrar', max_capas)
          layer1 = tf.keras.layers.Dense(cant_neuronas[max_capas], activation= tf.keras.layers.LeakyReLU(alpha=alpha[max_capas]))
          model.add(layer1)  
        else:
          #print('me hace bien', max_capas)
          model.add(Dense(cant_neuronas[max_capas], activation = activation_func[max_capas]))
        # Esta capa toma como el input el output de la primera capa oculta.
      max_capas += 1
  else:
    print("No coinciden la cantidad de capas con las dimensiones de 'cant_neuronas' y  'activation_func'")

  model.compile(optimizer= SGD(learning_rate= learning_rate), loss = loss, metrics = metrics)

  return model

In [ ]:
def crear_MLP_model(text_train, text_val, cant_capas, min_df_=0.000001, max_df_=0.8, ngram_range_ = (1,2), cant_neuronas = [3], activation_func = ["softmax"], learning_rate = 0.01, alpha = None, loss = "categorical_crossentropy", metrics = ["accuracy"]):
  # Conversión a sparse matrix:
  cv_train, cv_valid = count_vectorizer_MLP(text_train,text_val, min_df_, max_df_, ngram_range_)

  # Creación y compilación de Modelo deseado:
  model = MLP_model(cant_capas, cv_train, cant_neuronas= cant_neuronas, activation_func= activation_func, learning_rate= learning_rate, alpha= alpha, loss = loss, metrics= metrics)

  # El return:
  return model

In [ ]:
def entrenar_MLP_model(model,text_train, labels_train, text_val, labels_val, batch_size, batch_val, min_df_=0.000001, max_df_=0.8, ngram_range_ = (1,2), verbose = 1, epochs = 10):
  # Conversión a sparse matrix:
  cv_train, cv_valid = count_vectorizer_MLP(text_train,text_val, min_df_, max_df_, ngram_range_)
  labels_train_enc, labels_valid_enc = labels_transform(labels_train, labels_val)
  print(cv_train.shape)
  print(cv_valid.shape)

  # Entrenamiento del modelo:
  print('training model....')
  model.fit(DataGenerator(cv_train, labels_train_enc, batch_size), validation_data=DataGenerator(cv_valid,labels_valid_enc,batch_val),verbose= verbose, epochs = epochs)

  return model

In [ ]:
def count_vector_test(train, test, min_df_=0.000001, max_df_=0.8, ngram_range_ = (1,1)):
  cv = CountVectorizer(min_df=min_df_, max_df=max_df_, ngram_range = ngram_range_)
  cv_train = cv.fit_transform(train)
  cv_test = cv.transform(test)
  return cv_test

In [ ]:
def convert_pd(datos, labels):
  len_datos = datos.shape[0]
  list_predict = []
  for index in range(len_datos):
    list_predict.append([labels[datos[index]]])
  submission = pd.DataFrame(data=list_predict, columns=["pred_labels"])
  submission.index.names = ["pairID"]
  return submission

In [ ]:
def transform_string_num(list_labels):
  arg = np.zeros(len(list_labels))
  for i in range(len(arg)):
    if list_labels[i] == 'contradiction':
      arg[i] = int(0)
    elif list_labels[i] == 'entailment':
      arg[i] = int(1)
    elif list_labels[i] == 'neutral':
      arg[i] = int(2)
  return arg